In [1]:
import feedparser
import requests
from sqlalchemy import create_engine, Column, Integer, String, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [2]:
Base = declarative_base()
# Create the SQLAlchemy engine and bind it to the Flask app
class Video(Base):
    __tablename__ = 'videos'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    url = Column(String)
    thumbnail = Column(String)
    tags = Column(String)
    embed = Column(String)

/var/folders/r8/pvwq05bn6j942__l82_tjrkh0000gn/T/ipykernel_43916/2397935309.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
engine = create_engine('sqlite:///data.db')
Base.metadata.create_all(engine)


In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
url = 'http://www.pornhub.com/rss'
response = requests.get(url)

if response.status_code == 200:
    rss_feed = response.text

    # Parse the RSS feed
    feed = feedparser.parse(rss_feed)
    print(feed)
    # Filter entries containing the keyword 'mom'
    mom_entries = [
        entry for entry in feed.entries if 'mom' in entry.title.lower() or 'mom' in entry.tags ]

    # Print the number of filtered entries
    print(f"Number of entries containing 'mom': {len(mom_entries)}")

    # Print the titles of the filtered entries
    for entry in mom_entries:
        print(entry.title)
        print(entry.link)
        print(entry.thumb)
        print(entry.tags[0].term)
        print(entry.embed)
else:
    print(f'Failed to fetch RSS feed. Status code: {response.status_code}')


{'bozo': False, 'entries': [{'title': 'TEENGONZO Abella Danger gets her way', 'title_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': 'TEENGONZO Abella Danger gets her way'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.pornhub.com/view_video.php?viewkey=6581b4fdd409b'}], 'link': 'https://www.pornhub.com/view_video.php?viewkey=6581b4fdd409b', 'duration': '624', 'thumb': 'https://ei.phncdn.com/videos/202312/19/445051021/original/(m=q-L4J6Ybe0YHGgaaaa)(mh=sqFZ68Oq4UCocz2q)0.jpg', 'thumb_large': 'https://ei.phncdn.com/videos/202312/19/445051021/original/(m=q-L4J6Ybeaf8Ggaaaa)(mh=z46QfrOP7uirhunj)0.jpg', 'embed': '<iframe src="https://www.pornhub.com/embed/6581b4fdd409b" frameborder="0" width="560" height="340" scrolling="no" allowfullscreen></iframe>', 'tags': [{'term': 'teengonzo,abella danger,anal,blowjob,deepthroat,brunette,small tits,shaved pussy,heels,doggystyle,missionary,cowgirl,reverse cowgirl,ass fuck,anal sex,1080p', 'scheme': No

In [6]:
for entry in mom_entries:
    Video_Entry = Video(
        title=entry.title,
        url=entry.link,
        thumbnail=entry.thumb,
        tags=entry.tags[0].term,
        embed=entry.embed
    )
    session.add(Video_Entry)

# Commit the changes to the database
session.commit()